# Auxiliary Classifier with Generative Adversarial Network 

## Project Info
this notebook is an implementation for Auxiliary Classifier with Generative Adversarial Network 

# Header


 ## Import Necessary Packages

In [0]:
#Basic Level Libs
import math
import os 

#Mid Level Libs
import numpy as np

#FrameWorks
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable

import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import save_image

#Other
import matplotlib.pyplot as plt


## Mout Google Dirve

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Dataset 

In [15]:
batch_size_train = 64
batch_size_test = 1000

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ])),
    
  batch_size=batch_size_train, shuffle=True)

valid_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:01, 8899902.54it/s]                            


Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134341.57it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2210273.33it/s]                            
0it [00:00, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 50292.06it/s]            


Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## Displayers

## Data

In [0]:
def imshow(img):
    plt.imshow(np.transpose(img, (1, 2, 0)).squeeze())

In [0]:
def displaying_data(dataiter):
    # obtain one batch of training images
    images, labels = dataiter.next()
    images = images.numpy() # convert images to numpy for display

    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(25, 4))
    # display 20 images
    for idx in np.arange(20):
        ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
        try:
            imshow(images[idx][0])
        except:
            imshow(images[idx])

        ax.set_title(str(int(labels[idx])))

In [0]:
iterdata = iter(train_loader)
displaying_data(iterdata)

### Image saver

In [0]:
!rm -rf '/content/drive/My Drive/images/'
if not os.path.exists('/content/drive/My Drive/images/'):
    os.mkdir('/content/drive/My Drive/images/')
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    noise = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, embed_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = G(labels, noise)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


# Model

## Helpers

In [0]:
class View:
    def __init__(self, reshape):
        self.reshape = reshape

    def __call__(self, x):
        return x.view(*self.reshape)

## Generator

In [0]:
class Generator(nn.Module):
    def __init__(self,no_classes, em_dim, img_shape):
        super(Generator, self).__init__()

        c, h, w = img_shape
        init_img_size = h // 4
        self.embedding = nn.Embedding(num_embeddings=no_classes, embedding_dim= em_dim)
        self.linear = nn.Linear(em_dim, em_dim * init_img_size ** 2)
        self.relu = nn.ReLU()
        self.us = nn.Upsample(scale_factor=2)
        self.leakyrelu = nn.LeakyReLU(0.2)
        self.tanh = nn.Tanh()
        self.view = View((-1, em_dim, init_img_size, init_img_size))

        self.bn1 = nn.BatchNorm2d(em_dim, momentum=0.8)
        self.conv1 = nn.Conv2d(em_dim, 128, 3, 1, 1)

        self.bn2 = nn.BatchNorm2d(128, momentum=0.8)
        self.conv2 = nn.Conv2d(128, 64, 3, 1, 1)

        self.bn3 = nn.BatchNorm2d(64, momentum=0.8)
        self.conv3 = nn.Conv2d(64, 32, 3, 1, 1)

        self.bn4 = nn.BatchNorm2d(32, momentum=0.8)
        self.conv4 = nn.Conv2d(32, c, 3, 1, 1)

    def forward(self, labels, noise):
        
        x = self.embedding(labels)
        x = torch.mul(x, noise)
        x = self.linear(x)
        x = self.relu(x)
        x = self.view(x)

        x = self.bn1(x)
        x = self.us(x)
        x = self.conv1(x)

        x = self.bn2(x)
        x = self.leakyrelu(x)
        x = self.us(x)
        x = self.conv2(x)

        x = self.bn3(x)
        x = self.leakyrelu(x)
        x = self.conv3(x)
        
        x = self.bn4(x)
        x = self.leakyrelu(x)
        x = self.conv4(x)

        x = self.tanh(x)
        return x




## Discriminator

**Repeated block**

In [0]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, bool_bn=True):
        super(Block, self).__init__()
        
        self.bool_bn = bool_bn
        self.conv = nn.Conv2d(in_channels, out_channels, 3, 2, 1)
        self.leakyrelu = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout2d(0.25)

        if self.bool_bn:
            self.bn = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        if self.bool_bn:
            x = self.bn(x)
        return x


In [0]:
class Discriminator(nn.Module):
    def __init__(self, no_class, img_shape):
        super(Discriminator, self).__init__()
        c, h, w = img_shape
        
        self.block1 = Block(c, 16, bool_bn=False)
        self.block2 = Block(16, 32)
        self.block3 = Block(32, 64)
        self.block4 = Block(64, 128)

        downed_size = int(math.ceil(h / 2**4))

        self.view = View((-1, 128 * downed_size ** 2))

        self.advv_layer = nn.Linear(128 * downed_size ** 2 , 1)
        self.aux_layer = nn.Linear(128 * downed_size ** 2 , no_class)

        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()

    def forward(self, img):
        x = self.block1(img)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)   

        x = self.view(x)

        adv = self.advv_layer(x)
        adv = self.sigmoid(adv)

        aux = self.aux_layer(x)
        aux = self.softmax(aux)
        
        return adv, aux
        

# Workers

## Models, Optimizers & Losses

**Configs** 

In [0]:
#image shape
img, _ = iterdata.next()
image_shape = img[0].shape

#number of classes
n_classes = 10

#embedding dim
embed_dim = 128

# checking for GPU availability
cuda = torch.cuda.is_available()

#Learning rate
lr = 0.0002

#B1 & B2
b1 = 0.5
b2 = 0.999

#Path to save trained model
model_file = ''

# numper of epochs
n_epochs = 250

#Data types
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

#interval to save images
sample_interval = 100


**Models**

In [0]:
G = Generator(n_classes, embed_dim, image_shape)
D = Discriminator(n_classes, image_shape)

**Losses**  

In [0]:
adv_loss = torch.nn.BCELoss()
aux_loss = torch.nn.CrossEntropyLoss()

**Cuda** 

In [0]:
if cuda:
    print("Cuda is Available")
    G.cuda()
    D.cuda()

    adv_loss.cuda()
    aux_loss.cuda()

Cuda is Available


**Optimizers** 

In [0]:
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))


## Trainer

In [0]:
min_G_valid_loss = np.Inf
min_D_valid_loss = np.Inf
saved_epoch = 0
for epoch in range(n_epochs):
    G_valid_loss = 0
    D_valid_loss = 0
    for i, (imgs,labels) in enumerate(train_loader):
        
        batch_size = imgs.shape[0]
        
        real = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        gen = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)    
        
        real_labels = Variable(labels.type(LongTensor))
        real_imgs = Variable(imgs.type(FloatTensor))

        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))
        noise = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, embed_dim))))


        optimizer_G.zero_grad()

        gen_imgs = G(gen_labels, noise)

        adv_score, aux_score = D(gen_imgs)

        G_loss = (adv_loss(adv_score, real) + aux_loss(aux_score, gen_labels)) / 2.
        
        G_loss.backward()
        optimizer_G.step()
        
        optimizer_D.zero_grad()

        real_adv_score, real_aux_score = D(real_imgs)
        real_loss = (adv_loss(real_adv_score, real) + aux_loss(real_aux_score, real_labels)) / 2

        gen_adv_score, gen_aux_score = D(gen_imgs.detach())
        gen_loss = (adv_loss(gen_adv_score, gen) + aux_loss(gen_aux_score, gen_labels)) / 2

        D_loss = (real_loss + gen_loss) / 2.
        
        D_loss.backward()
        optimizer_D.step()

        pred = np.concatenate([real_aux_score.data.cpu().numpy(), gen_aux_score.data.cpu().numpy()], axis=0)
        gt = np.concatenate([real_labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        print(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, n_epochs, i, len(train_loader), D_loss.item(), 100 * d_acc, G_loss.item()), end=''
        )
       
        batches_done = epoch * len(train_loader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)
        
        G_valid_loss += G_loss.item()
        D_valid_loss += D_loss.item()

    G_valid_loss /= len(train_loader)
    D_valid_loss /= len(train_loader)
            
    if min_G_valid_loss > G_valid_loss:
        print ('\nG Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...\n'.format(min_G_valid_loss, G_valid_loss))
        min_valid_loss = G_valid_loss
        torch.save(G.state_dict(), '/content/drive/My Drive/Models/G.pth')
        saved_epoch = epoch
    if min_D_valid_loss > D_valid_loss:
        print ('\nD Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...\n'.format(min_D_valid_loss, D_valid_loss))
        min_D_valid_loss = D_valid_loss
        torch.save(D.state_dict(), '/content/drive/My Drive/Models/D.pht')
    if epoch - saved_epoch > 15:
        print("Early Stopping")
        break


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 0/250] [Batch 937/938] [D loss: 1.256521, acc: 60%] [G loss: 1.400350]
G Validation loss decreased (inf --> 1464.384704).         Saving model ...


D Validation loss decreased (inf --> 1255.824039).         Saving model ...

[Epoch 1/250] [Batch 937/938] [D loss: 1.096992, acc: 73%] [G loss: 1.365475]
G Validation loss decreased (inf --> 1384.953418).         Saving model ...


D Validation loss decreased (1255.824039 --> 1125.678148).         Saving model ...

[Epoch 2/250] [Batch 937/938] [D loss: 1.222710, acc: 78%] [G loss: 1.321601]
G Validation loss decreased (inf --> 1331.380878).         Saving model ...


D Validation loss decreased (1125.678148 --> 1059.269634).         Saving model ...

[Epoch 3/250] [Batch 937/938] [D loss: 1.091178, acc: 81%] [G loss: 1.257431]
G Validation loss decreased (inf --> 1298.123950).         Saving model ...


D Validation loss decreased (1059.269634 --> 1033.413515).         Saving model ...

[Epoch 4/250] [Batch 937/938] [D loss: 1.025

KeyboardInterrupt: ignored

## GIF

In [0]:
import glob
import imageio

In [0]:
with imageio.get_writer('AC_GAN.gif', mode='I') as writer:
  
  files = os.listdir('/content/drive/My Drive/images')
  new_files = [int(f[:-4]) for f in files]
  filenames = ['/content/drive/My Drive/images/' + str(f)+'.png' for f in new_files]

  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# this is a hack to display the gif inside the notebook
os.system('cp dcgan.gif AC_GAN.gif.png')

256